In [1]:
import pandas as pd
import os

In [2]:
normal_file_path = '/Users/shreyamantripragada/Documents/rise2021/data/normal'
schizophrenia_file_path = '/Users/shreyamantripragada/Documents/rise2021/data/schizophrenia'

In [7]:
def reshape_data (file_path): 
    '''
        Given the file_path of the folder, loop through the files within the folder and 
        return a combined dataframe consisting of EEG amplitudes for each patient within a single row.
        
        @param      file_path 
        
    '''
    new_df = pd.DataFrame() # create a new, empty data frame
    filenames = [] # create an empty list which will eventually contain the file names
    
    for filename in os.listdir(file_path): # loop through the files within the folder 
        new_df = new_df.append(pd.read_csv(os.path.join(file_path, filename)).transpose()) # append the values of each file into the data frame
        filenames.append(os.path.splitext(filename)[0]) # append the file names to the list 'filenames'
        
    first_row_values = new_df.index # get the row names and the 
    new_df.insert(loc = 0, column = '0', value = first_row_values) # push the current row indices to the first column
    new_df = new_df.T.reset_index(drop=True).T # reset the column indices
    new_df.index = filenames # reassign the new row indices to the filenames
    
    return new_df # return the new, combined data frame

In [9]:
normal_data = reshape_data(normal_file_path)
schizo_data = reshape_data(schizophrenia_file_path)

In [10]:
normal_data

,0,1,2,3,4,5,6,7,8,9,...,122870,122871,122872,122873,122874,122875,122876,122877,122878,122879
S42W1,-154.12,-112.09,-154.12,-154.12,-14.01,-51.37,-233.51,-233.51,-112.09,-112.09,...,18.68,317.58,616.47,219.5,-233.51,-32.69,158.79,177.47,317.58,126.1
S31W,308.45,248.96,328.28,429.63,469.29,429.63,508.94,627.92,887.9,808.58,...,-262.18,-321.67,-420.82,-491.32,-581.65,-550.81,-451.66,-370.14,-121.18,-50.67
S167W1,-91.45,-91.45,-150.39,-231.68,-191.03,-111.77,-30.48,-91.45,-270.29,-331.26,...,239.8,138.19,-50.81,-231.68,-591.38,-731.61,-731.61,-611.7,-270.29,-40.64
S50W,248.66,229.26,229.26,308.62,449.71,409.15,128.74,29.98,149.9,209.86,...,-21.16,-430.31,-1010.52,-1171.0,-1331.49,-1331.49,-1070.48,-751.28,98.76,619.01
S165W1,8.95,-10.07,-170.04,-250.59,-350.16,-330.02,-151.03,-50.34,8.95,-50.34,...,-30.21,-130.89,-350.16,-390.43,-210.32,-30.21,219.27,279.68,199.13,99.57
s158w1,68.85,68.85,109.21,89.03,-51.05,-170.94,-311.02,-290.84,-170.94,-110.4,...,-490.27,299.15,679.02,699.2,499.77,339.51,39.17,-51.05,-330.01,-451.1
s178w1,-350.69,-250.14,-131.2,-50.27,169.21,329.84,529.71,549.33,589.79,609.41,...,519.9,378.89,159.4,79.7,58.86,79.7,9.81,-141.01,-440.2,-570.18
S18W1,26.43,147.27,207.69,86.85,-30.21,-11.33,169.93,226.58,67.97,-52.87,...,226.58,117.06,56.64,79.3,128.39,139.72,147.27,139.72,237.9,317.21
s27w1,109.39,129.14,-150.41,-250.68,-150.41,-130.66,-350.96,-510.48,-370.71,-110.91,...,98.75,218.78,598.6,679.13,499.85,399.57,218.78,118.51,0.0,19.75
S163W1,309.04,349.67,388.84,388.84,468.64,488.95,429.47,349.67,208.93,169.76,...,139.29,18.86,-190.07,-271.32,-240.85,-181.36,-60.94,-50.78,0.0,49.33


In [11]:
schizo_data

,0,1,2,3,4,5,6,7,8,9,...,122870,122871,122872,122873,122874,122875,122876,122877,122878,122879
312w1,-71.15,9.7,148.76,129.36,48.51,9.7,29.11,48.51,189.19,289.44,...,858.63,1039.73,1099.56,999.31,489.95,38.81,-981.52,-1460.15,-1930.7,-1961.42
642w1,189.75,505.99,965.98,908.48,230.0,-132.25,-252.99,28.75,724.48,707.23,...,-11.5,258.74,574.99,597.99,494.49,396.74,178.25,138.0,138.0,138.0
387-02w1,-30.80,24.64,49.28,86.24,104.72,67.76,6.16,-12.32,-55.44,-55.44,...,-412.74,-154.01,277.21,314.17,197.13,234.09,498.98,597.54,579.06,498.98
s351w,-50.11,108.94,229.86,229.86,229.86,229.86,229.86,149.25,29.41,-50.11,...,-100.22,-410.7,-530.54,-350.79,-70.81,-10.89,69.72,259.28,469.53,589.36
423w,-34.91,-93.09,-81.46,-34.91,0.0,23.27,23.27,23.27,0.0,-11.64,...,314.19,314.19,209.46,139.64,34.91,0.0,-116.37,-116.37,11.64,93.09
485w1,-10.19,-130.38,-291.31,-270.94,8.15,107.97,169.08,69.26,-71.3,-91.67,...,-171.12,-330.01,-751.7,-1010.42,-1370.99,-1411.73,-1110.23,-790.41,8.15,338.16
314w1,-11.27,7.51,-71.38,-131.49,-112.7,-11.27,108.94,86.4,-90.16,-191.59,...,-942.94,-1021.83,-822.72,-601.07,-71.38,116.46,608.59,706.26,518.43,229.16
585w1,-230.05,-211.2,-50.91,28.29,88.63,69.77,-11.31,9.43,109.37,128.23,...,-931.53,-1110.67,-1171.01,-971.13,9.43,469.54,1129.53,1229.47,869.3,569.48
113w1,-35.09,-35.09,-52.63,-93.56,-111.1,-93.56,-11.7,5.85,87.71,87.71,...,315.77,619.84,1035.02,1099.35,877.14,654.93,116.95,-93.56,-450.26,-532.13
156w1,-111.96,-150.2,-191.16,-172.04,-30.04,8.19,27.31,68.27,128.35,128.35,...,-412.35,-450.59,-352.28,-251.24,19.12,237.58,658.13,778.29,778.29,679.98
